In [25]:
import numpy as np
import matplotlib.pyplot as plt
import wave
import struct
import time
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
from scipy.fft import fft
import cupy as cp
from scipy.io import wavfile

In [34]:
# Функция для чтения WAV файла
def read_wav(filename):
    framerate, audio_data = wavfile.read(filename)

     # Если данные стерео, преобразуем их в моно
    if len(audio_data.shape) == 2:
        audio_data = np.mean(audio_data, axis=1)

    return framerate, audio_data

# Функция для вычисления спектрограммы с использованием FFTW (CPU)
def compute_spectrogram_fftw(audio_data, framerate, window_size=1024, overlap=512):
    n_chunks = int(np.ceil(len(audio_data) / float(overlap)))
    spectrogram = []
    
    for i in range(n_chunks):
        start = i * overlap
        end = start + window_size
        chunk = audio_data[start:end]
        
        if len(chunk) < window_size:
            chunk = np.pad(chunk, (0, window_size - len(chunk)), 'constant')
        
        # Применяем окно Хэмминга
        chunk = chunk * np.hamming(window_size)
        
        # Преобразование на CPU
        spec = np.fft.fft(chunk)
        
        # Получаем спектр
        spec = np.abs(spec[:window_size // 2]) ** 2
        spectrogram.append(spec)
    
    return np.array(spectrogram).T  # Транспонируем для правильного отображения

# Функция для вычисления спектрограммы с использованием CUFFT (GPU)
def compute_spectrogram_cufft(audio_data, framerate, window_size=1024, overlap=512):
    n_chunks = int(np.ceil(len(audio_data) / float(overlap)))
    spectrogram = []
    
    for i in range(n_chunks):
        start = i * overlap
        end = start + window_size
        chunk = audio_data[start:end]
        
        if len(chunk) < window_size:
            chunk = np.pad(chunk, (0, window_size - len(chunk)), 'constant')
        
        # Применяем окно Хэмминга
        chunk = chunk * np.hamming(window_size)
        
        # Переносим данные на GPU
        d_chunk = cp.asarray(chunk)  # Конвертируем в cupy.ndarray
        
        # Преобразование на GPU
        d_spec = cp.fft.fft(d_chunk)  # Выполняем FFT на GPU
        
        # Получаем спектр
        spec = cp.abs(d_spec[:window_size // 2]) ** 2
        spectrogram.append(cp.asnumpy(spec))
    
    return np.array(spectrogram).T  # Транспонируем для правильного отображения

# Функция для сохранения спектрограммы
def save_spectrogram(spectrogram, filename):
    plt.imshow(spectrogram, aspect='auto', cmap='hot', origin='lower')
    plt.colorbar()
    plt.savefig(filename, format='png')
    plt.close()

# Основная функция
def main():
    framerate, audio_data = read_wav('input.wav')
    
    # Вычисление спектрограммы с использованием FFTW (CPU)
    start_time = time.time()
    spectrogram_fftw = compute_spectrogram_fftw(audio_data, framerate)
    cpu_time = time.time() - start_time
    print(f"Время вычисления спектрограммы (FFTW, CPU): {cpu_time} секунд")
    
    # Вычисление спектрограммы с использованием CUFFT (GPU)
    start_time = time.time()
    spectrogram_cufft = compute_spectrogram_cufft(audio_data, framerate)
    gpu_time = time.time() - start_time
    print(f"Время вычисления спектрограммы (CUFFT, GPU): {gpu_time} секунд")
    
    # Сохранение спектрограммы в файл
    save_spectrogram(spectrogram_fftw, 'spectrogram_cpu.png')
    save_spectrogram(spectrogram_cufft, 'spectrogram_gpu.png')

In [ ]:
import numpy as np
from scipy.io.wavfile import write

# Параметры аудио
rate = 44100  # Частота дискретизации
T = 6  # Длительность в секундах
f = 440  # Частота сигнала (Гц)

# Генерация сигнала
t = np.linspace(0, T, int(rate * T), endpoint=False)
signal = np.sin(2 * np.pi * f * t)

# Преобразуем в 16-битный формат и сохраняем
signal = np.int16(signal * 32767)  # Преобразуем в 16-битный формат

# Сохраняем в WAV файл
write('input.wav', rate, signal)

In [35]:
# Запуск программы
if __name__ == '__main__':
    main()

Время вычисления спектрограммы (FFTW, CPU): 0.025018692016601562 секунд
Время вычисления спектрограммы (CUFFT, GPU): 0.43738770484924316 секунд
